In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from utils.metrics import mar, far, f1_score

from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector

In [14]:
df = pd.read_csv("./data/valve1/0.csv", sep=";", parse_dates=True, index_col="datetime")

df.sort_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1147 entries, 2020-03-09 10:14:33 to 2020-03-09 10:34:32
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Accelerometer1RMS    1147 non-null   float64
 1   Accelerometer2RMS    1147 non-null   float64
 2   Current              1147 non-null   float64
 3   Pressure             1147 non-null   float64
 4   Temperature          1147 non-null   float64
 5   Thermocouple         1147 non-null   float64
 6   Voltage              1147 non-null   float64
 7   Volume Flow RateRMS  1147 non-null   float64
 8   anomaly              1147 non-null   float64
 9   changepoint          1147 non-null   float64
dtypes: float64(10)
memory usage: 98.6 KB


In [15]:
from utils.data import load_df

In [16]:
train_size = 400

In [17]:
SS = StandardScaler()
x_train = np.array(SS.fit_transform(df[:train_size].drop(['anomaly','changepoint'], axis=1)))

In [18]:
TIME_STEPS = 50

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

x_train = create_sequences(x_train)
print("Training input shape: ", x_train.shape)

Training input shape:  (351, 50, 8)


In [19]:
n_features = x_train.shape[-1]
window_shape = (TIME_STEPS, n_features)
def get_model():   
    encoder_input = Input(shape=window_shape)
    encoder_LSTM = LSTM(128, return_state = True)
    encoder_outputs, encoder_h, encoder_c = encoder_LSTM(encoder_input)
    encoder_states = [encoder_h, encoder_c]
    
    decoder_input = Input(shape = window_shape)
    decoder_LSTM = LSTM(128, return_sequence = True,
                        return_state = True)
    decoder_out, _, _ = decoder_LSTM(decoder_input, initial_state = encoder_states)#h & c are not returned
    decoder_dense = TimeDistributed(Dense(n_features, activation = "relu"))